In [ ]:
!pip install pandas pyarrow

In [24]:
import pandas as pd
import numpy as np

from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

from stepmix.stepmix import StepMix

In [2]:
# data2004_f = pd.read_parquet("data/data2004_f.parquet")
data2004_i = pd.read_parquet("data/data2004_i.parquet")

In [4]:
data = data2004_i[
    ['clseusa_n', 'ambornin_n', 'amcit_n', 'amlived_n', 'amenglsh_n', 
     'amchrstn_n', 'amgovt_n', 'amfeel_n', 'amcitizn_n', 'amshamed_n', 
     'belikeus_n', 'ambetter_n', 'ifwrong_n', 'proudsss_n', 'proudgrp_n', 
     'proudpol_n', 'prouddem_n', 'proudeco_n', 'proudspt_n', 'proudart_n', 
     'proudhis_n', 'proudmil_n', 'proudsci_n']
]

In [ ]:
results = []

for n in range(2, 30):
    model = StepMix(n_components=n, 
                    measurement='continuous', 
                    n_init=5)
    model.fit(data)
    labels = model.predict(data
    
    results.append({
        'n_components': n,
        'aic': model.aic(data),
        'bic': model.bic(data),
        'silhouette': silhouette_score(data, labels),
        'calinski_harabasz': calinski_harabasz_score(data, labels),
        'davies_bouldin': davies_bouldin_score(data, labels)
    })

results = pd.DataFrame(results)

In [27]:
results

,n_components,aic,bic,silhouette,calinski_harabasz,davies_bouldin
0,2,52229.028071,52703.560511,0.150056,145.528168,2.717648
1,3,47178.759340,47893.109250,0.081011,126.050859,2.937177
2,4,34999.180047,35953.347426,0.055459,91.809426,4.610678
3,5,31971.008248,33164.993098,0.045137,70.581351,4.179338
4,6,31238.231272,32672.033591,0.006370,56.904937,4.050324
5,7,29064.257206,30737.876995,0.002942,49.491833,4.330852
6,8,19152.114119,21065.551377,0.028070,49.246828,4.959759
7,9,14305.050393,16458.305121,-0.002209,45.907881,4.569558
8,10,17528.924478,19921.996676,0.015968,43.046850,4.260373
9,11,16401.712890,19034.602557,-0.013546,35.343173,4.694781
